# Create interpolated dictionaries

#### Author: Megan Thompson-Munson
#### Date created: 20 September 2021

This script reads in the dictionaries of SUMup and SNOWPACK data and interpolates to a common vertical scale. The interpolated data is then saved in a dictionary as a pickle file.

TO DO:
- Add more comments
- Add CFM capability
- Move analysis part to new file

In [19]:
icesheet = 'GrIS'

In [20]:
# Import libraries
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
from scipy.interpolate import griddata

In [21]:
data = pickle.load(open(icesheet+'_data.p','rb'))

In [22]:
# List of dictionaries
dict_list = []

# Regrid observations and model output on logarithmic vertical scale
for i in range(len(data)):

    # Select SUMup and SNOWPACK keys
    su = data[i]['SUMup']
    sp = data[i]['SNOWPACK']
    
    # Define relevant variables
    sp_depth = sp['Depth']
    sp_density = sp['Density']
    sp_air = sp['Air']
    sp_water = sp['Water']
    sp_ice = sp['Ice']
    su_density = su['Density']
    su_depth = su['Midpoint']
    
    # Many SUMup measurements are just single point, so just regrid datasets with n > 1
    if len(su_density) > 1:
        
        # Interpolate modeled density onto observations
        su_densityInt = su_density
        sp_densityInt = griddata(sp_depth,sp_density,su_depth)
        
        # Filter out nans
        nanfilter = (~np.isnan(su_densityInt)) & (~np.isnan(sp_densityInt))
        su_densityFilt = su_densityInt[nanfilter]
        sp_densityFilt = sp_densityInt[nanfilter]
        
        dictionaries = {'SNOWPACK':{'ID':sp['ID'],'Timestamp':sp['Timestamp'],
                                    'Latitude':sp['Latitude'],'Longitude':sp['Longitude'],
                                    'Elevation':sp['Elevation'],'Depth':su_depth,'Density':sp_densityFilt},
                        'SUMup':{'CoreID':su['CoreID'],'Citation':su['Citation'],'Timestamp':su['Timestamp'],
                                 'Latitude':su['Latitude'],'Longitude':su['Longitude'],
                                 'Elevation':su['Elevation'],'Depth':su_depth,'Density':su_densityFilt}}

        dict_list.append(dictionaries)

In [23]:
pickle.dump(dict_list, open(icesheet+'_data_interpolated.p','wb'))